<a href="https://colab.research.google.com/github/RealMyeong/Aiffel_Exploration/blob/main/EX_BS2_project14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 14. 프로젝트 : 한국어 데이터로 챗봇 만들기


## Step 1. 데이터 수집하기

In [1]:
# 필요한 모듈 임포트 하기
import tensorflow as tf
import numpy as np
import re
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_datasets as tfds

## Step 2. 데이터 전처리하기

- 데이터 불러오기
  - Data description.
    - 챗봇 트레이닝용 문답 페어 11,876개
    - 일상다반사 0, 이별(부정) 1, 사랑(긍정) 2로 레이블링

In [2]:
data_path = '/content/drive/MyDrive/AIFFEL/EX/14_chatbot/project/ChatbotData .csv'
data = pd.read_csv(data_path)
data.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [3]:
data.tail(10)

,Q,A,label
11813,회사에 좋아하는 남자가 생겼어 어떡하지?,사랑하기 힘든 관계인가봐요.,2
11814,회사에서 어떤 사람이랑 자꾸 눈 마추쳐.,눈 마주치는 게 우연인지 잘 살펴 보세요.,2
11815,회식 중이라고 하는데 연락이 안돼.,정신 없이 바쁠지도 몰라요. 조금만 더 기다려보고 물어보는게 좋을 것 같아요.,2
11816,회식하는데 나만 챙겨줘. 썸임?,호감이 있을 수도 있어요. 그렇지만 조금 더 상황을 지켜보세요.,2
11817,후회 없이 사랑하고 싶어,진심으로 다가가 보세요.,2
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2
11822,힘들어서 결혼할까봐,도피성 결혼은 하지 않길 바라요.,2


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11823 entries, 0 to 11822
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Q       11823 non-null  object
 1   A       11823 non-null  object
 2   label   11823 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 277.2+ KB


- 결측치 확인

In [5]:
data.isnull().sum()

Q        0
A        0
label    0
dtype: int64

- 전처리 함수 정의
  - 구두점 제거


In [6]:
# 전처리 함수
def preprocess_sentence(sentence):
  sentence = sentence.strip()

  # 단어와 구두점 사이의 거리를 만듦
  sentence = re.sub(r"([?.!,~])", r" \1", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)

  # (ㄱ-ㅎ, ㅏ-ㅣ, 가-힣, '.', '?', '!', ',')를 제외한 모든 문자를 공백인 ' '로 대체합니다
  sentence = re.sub(r"[^가-힣?.!,]+", " ", sentence)
  sentence = sentence.strip()
  return sentence

Q, A라벨 별로 전처리 함수 적용해서 전처리 진행해줌

In [7]:
data['Q'] = data['Q'].apply(lambda x : preprocess_sentence(x))
data['A'] = data['A'].apply(lambda x : preprocess_sentence(x))

In [8]:
data.head(10)

,Q,A,label
0,시 땡 !,하루가 또 가네요 .,0
1,지망 학교 떨어졌어,위로해 드립니다 .,0
2,박 일 놀러가고 싶다,여행은 언제나 좋죠 .,0
3,박 일 정도 놀러가고 싶다,여행은 언제나 좋죠 .,0
4,심하네,눈살이 찌푸려지죠 .,0
5,카드 망가졌어,다시 새로 사는 게 마음 편해요 .,0
6,카드 안돼,다시 새로 사는 게 마음 편해요 .,0
7,맞팔 왜 안하지,잘 모르고 있을 수도 있어요 .,0
8,시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요 .,0
9,시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요 .,0


In [9]:
data.tail(10)

,Q,A,label
11813,회사에 좋아하는 남자가 생겼어 어떡하지 ?,사랑하기 힘든 관계인가봐요 .,2
11814,회사에서 어떤 사람이랑 자꾸 눈 마추쳐 .,눈 마주치는 게 우연인지 잘 살펴 보세요 .,2
11815,회식 중이라고 하는데 연락이 안돼 .,정신 없이 바쁠지도 몰라요 . 조금만 더 기다려보고 물어보는게 좋을 것 같아요 .,2
11816,회식하는데 나만 챙겨줘 . 썸임 ?,호감이 있을 수도 있어요 . 그렇지만 조금 더 상황을 지켜보세요 .,2
11817,후회 없이 사랑하고 싶어,진심으로 다가가 보세요 .,2
11818,훔쳐보는 것도 눈치 보임 .,티가 나니까 눈치가 보이는 거죠 !,2
11819,훔쳐보는 것도 눈치 보임 .,훔쳐보는 거 티나나봐요 .,2
11820,흑기사 해주는 짝남 .,설렜겠어요 .,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까 ?,잘 헤어질 수 있는 사이 여부인 거 같아요 .,2
11822,힘들어서 결혼할까봐,도피성 결혼은 하지 않길 바라요 .,2


전처리가 잘 된 것을 볼 수 있음

데이터에서 라벨은 딱히 상관이 없어 보인다.

 라벨 부분은 지우고 Q, A만 사용

In [10]:
data.drop('label', axis=1, inplace=True)

In [11]:
data.head()

,Q,A
0,시 땡 !,하루가 또 가네요 .
1,지망 학교 떨어졌어,위로해 드립니다 .
2,박 일 놀러가고 싶다,여행은 언제나 좋죠 .
3,박 일 정도 놀러가고 싶다,여행은 언제나 좋죠 .
4,심하네,눈살이 찌푸려지죠 .


In [12]:
# 질문, 답변 리스트 생성
question = list(data['Q'])
answer = list(data['A'])

## Step 3. SubwordTextEncoder 사용하기

한국어 데이터는 형태소 분석기를 사용하여 토크나이징을 해야 한다고 많은 분이 알고 있습니다. 하지만 여기서는 형태소 분석기가 아닌 위 실습에서 사용했던 내부 단어 토크나이저인 SubwordTextEncoder를 그대로 사용해보세요.

In [13]:
# 질문과 답변 데이터에 대해 Vacabulary 생성
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(question + answer, target_vocab_size=2**13)


.subwords 를 통해 토큰화된 서브워드 확인

In [14]:
print(tokenizer.subwords[:20])

[' .', ' ?', '거예요', '수_', '게_', '너무_', '더_', '거_', '는_', '좋아하는_', '이_', '을_', '잘_', '도_', ' . ', '고_', '요', '것_', '많이_', '안_']


원래 문장과 정수 인코딩 진행한 결과 확인

In [15]:
print(question[10])
a = tokenizer.encode(question[10])
print(f'정수 인코딩 결과 : {a}')
print(f'다시 디코딩 : {tokenizer.decode(a)}')

보면 나만 빼고 다 행복해보여
정수 인코딩 결과 : [485, 196, 2967, 45, 3805]
다시 디코딩 : 보면 나만 빼고 다 행복해보여


단어장 크기 확인

In [16]:
tokenizer.vocab_size

8127

각 단어와 매핑된 정수 확인

In [17]:
for ts in a:
  print ('{} ----> {}'.format(ts, tokenizer.decode([ts])))

485 ----> 보면 
196 ----> 나만 
2967 ----> 빼고 
45 ----> 다 
3805 ----> 행복해보여


시작 토큰과 종료 토큰도 단어장에 추가하여 정수 인코딩 진행

In [18]:
# 시작 토큰과 종료 토큰에 고유한 정수 부여
start_token = [tokenizer.vocab_size]
end_token = [tokenizer.vocab_size + 1]
print(f'시작 토큰 번호: {start_token}, 종료 토큰 번호 : {end_token}')

시작 토큰 번호: [8127], 종료 토큰 번호 : [8128]


In [19]:
# vacab_size 시작, 종료 토큰 고려하여 재설정
vocab_size = tokenizer.vocab_size + 2

패딩 진행

In [20]:
# 질문 데이터 길이들
q_word_len = [len(x) for x in question ]
# 답변 데이터 길이들
a_word_len = [len(x) for x in answer ]

# 각각 최대 길이 뽑아봄
print(f'질문 최대 길이 : {max(q_word_len)} \n답변 최대 길이 : {max(a_word_len)}')

질문 최대 길이 : 57 
답변 최대 길이 : 78


In [21]:
# 샘플의 최대 허용 길이 또는 패딩 후의 최종 길이
MAX_LENGTH = 60 

# 정수 인코딩, 최대 길이를 초과하는 샘플 제거, 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # 정수 인코딩 과정에서 시작 토큰과 종료 토큰을 추가
    sentence1 = start_token + tokenizer.encode(sentence1) + end_token
    sentence2 = start_token + tokenizer.encode(sentence2) + end_token

    # MAX_LENGTH 이하인 경우에만 데이터로 사용
    if len(sentence1) <= MAX_LENGTH and len(sentence2) <= MAX_LENGTH:
      tokenized_inputs.append(sentence1)
      tokenized_outputs.append(sentence2)

  # MAX_LENGTH로 모든 데이터셋을 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs

In [22]:
questions, answers = tokenize_and_filter(question, answer)
print(f'단어장의 크기 : {vocab_size}')
print(f'필터링 후의 질문 샘플 개수 : {len(questions)}')
print(f'필터링 후의 답변 샘플 개수 : {len(answers)}')

단어장의 크기 : 8129
필터링 후의 질문 샘플 개수 : 11823
필터링 후의 답변 샘플 개수 : 11823


질문과 답변의 쌍을 tf.data.Dataset API의 입력으로 사용하여 파이프라인을 구성합니다. 이 때, 교사 강요를 위해서 answer[:, :-1]를 디코더의 입력값, answer[:, 1:]를 디코더의 레이블로 사용합니다.

In [23]:
batch_size = 64
buffer_size = 11823

# 디코더는 이전의 target을 다음의 input으로 사용합니다.
# 따라서 outputs에서는 start_token을 제거하겠습니다.
dataset = tf.data.Dataset.from_tensor_slices(({
    'inputs' : questions,
    'dec_inputs' : answers[:, :-1]
},{
    'outputs' : answers[:, 1:]
}))

dataset = dataset.cache()
dataset = dataset.shuffle(buffer_size)
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)


## Step 4. 모델 구성하기

실습 내용을 참고하여 트랜스포머 모델을 구현합니다.

In [24]:
# 포지셔널 인코딩 레이어
class PositionalEncoding(tf.keras.layers.Layer):

  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2 * (i // 2)) / tf.cast(d_model, tf.float32))
    return position * angles

  def positional_encoding(self, position, d_model):
    # 각도 배열 생성
    angle_rads = self.get_angles(
        position=tf.range(position, dtype=tf.float32)[:, tf.newaxis],
        i=tf.range(d_model, dtype=tf.float32)[tf.newaxis, :],
        d_model=d_model)

    # 배열의 짝수 인덱스에는 sin 함수 적용
    sines = tf.math.sin(angle_rads[:, 0::2])
    # 배열의 홀수 인덱스에는 cosine 함수 적용
    cosines = tf.math.cos(angle_rads[:, 1::2])

    # sin과 cosine이 교차되도록 재배열
    pos_encoding = tf.stack([sines, cosines], axis=0)
    pos_encoding = tf.transpose(pos_encoding,[1, 2, 0]) 
    pos_encoding = tf.reshape(pos_encoding, [position, d_model])

    pos_encoding = pos_encoding[tf.newaxis, ...]
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:, :tf.shape(inputs)[1], :]


# 스케일드 닷 프로덕트 어텐션 함수
def scaled_dot_product_attention(query, key, value, mask):
  # 어텐션 가중치는 Q와 K의 닷 프로덕트
  matmul_qk = tf.matmul(query, key, transpose_b=True)

  # 가중치를 정규화
  depth = tf.cast(tf.shape(key)[-1], tf.float32)
  logits = matmul_qk / tf.math.sqrt(depth)

  # 패딩에 마스크 추가
  if mask is not None:
    logits += (mask * -1e9)

  # softmax적용
  attention_weights = tf.nn.softmax(logits, axis=-1)

  # 최종 어텐션은 가중치와 V의 닷 프로덕트
  output = tf.matmul(attention_weights, value)
  return output


class MultiHeadAttention(tf.keras.layers.Layer):

  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(
        inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0, 2, 1, 3])

  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs[
        'value'], inputs['mask']
    batch_size = tf.shape(query)[0]

    # Q, K, V에 각각 Dense를 적용합니다
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    # 병렬 연산을 위한 머리를 여러 개 만듭니다
    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    # 스케일드 닷 프로덕트 어텐션 함수
    scaled_attention = scaled_dot_product_attention(query, key, value, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

    # 어텐션 연산 후에 각 결과를 다시 연결(concatenate)합니다
    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))

    # 최종 결과에도 Dense를 한 번 더 적용합니다
    outputs = self.dense(concat_attention)

    return outputs


# 패딩 마스크
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x, 0), tf.float32)
  # (batch_size, 1, 1, sequence length)
  return mask[:, tf.newaxis, tf.newaxis, :]


# 룩어헤드 마스크
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len, seq_len)), -1, 0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)


# 인코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 두 개의 서브 레이어가 존재합니다.
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })

  # 어텐션의 결과는 Dropout과 Layer Normalization이라는 훈련을 돕는 테크닉을 수행
  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

  # 두 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)


# 인코더 층 쌓아서 인코더 만들기
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  # 패딩 마스크 사용
  padding_mask = tf.keras.Input(shape=(1, 1, None), name="padding_mask")

  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  # num_layers만큼 쌓아올린 인코더의 층.
  for i in range(num_layers):
    outputs = encoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name="encoder_layer_{}".format(i),
    )([outputs, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

# 디코더 하나의 레이어를 함수로 구현.
# 이 하나의 레이어 안에는 세 개의 서브 레이어가 존재합니다.
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None, d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None, d_model), name="encoder_outputs")
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')

  # 첫 번째 서브 레이어 : 멀티 헤드 어텐션 수행 (셀프 어텐션)
  attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })

  # 멀티 헤드 어텐션의 결과는 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention1 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

  # 두 번째 서브 레이어 : 마스크드 멀티 헤드 어텐션 수행 (인코더-디코더 어텐션)
  attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })

  # 마스크드 멀티 헤드 어텐션의 결과는
  # Dropout과 LayerNormalization이라는 훈련을 돕는 테크닉을 수행
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

  # 세 번째 서브 레이어 : 2개의 완전연결층
  outputs = tf.keras.layers.Dense(units=units, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  # 완전연결층의 결과는 Dropout과 LayerNormalization 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)


# 디코더 층을 쌓아서 디코더 만들기
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None, d_model), name='encoder_outputs')
  look_ahead_mask = tf.keras.Input(
      shape=(1, None, None), name='look_ahead_mask')

  # 패딩 마스크
  padding_mask = tf.keras.Input(shape=(1, 1, None), name='padding_mask')
  
  # 임베딩 레이어
  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))

  # 포지셔널 인코딩
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)

  # Dropout이라는 훈련을 돕는 테크닉을 수행
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(
        units=units,
        d_model=d_model,
        num_heads=num_heads,
        dropout=dropout,
        name='decoder_layer_{}'.format(i),
    )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

  return tf.keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)


- 트랜스포머 함수 정의

In [25]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")
  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  # 인코더에서 패딩을 위한 마스크
  enc_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

  # 디코더에서 미래의 토큰을 마스크 하기 위해서 사용합니다.
  # 내부적으로 패딩 마스크도 포함되어져 있습니다.
  look_ahead_mask = tf.keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

  # 두 번째 어텐션 블록에서 인코더의 벡터들을 마스킹
  # 디코더에서 패딩을 위한 마스크
  dec_padding_mask = tf.keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

  # 인코더
  enc_outputs = encoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout
  )(inputs=[inputs, enc_padding_mask])

  # 디코더
  dec_outputs = decoder(
      vocab_size=vocab_size,
      num_layers=num_layers,
      units=units,
      d_model=d_model,
      num_heads=num_heads,
      dropout=dropout
  )(inputs=[dec_inputs, enc_outputs, look_ahead_mask, dec_padding_mask])

  # 완전연결층
  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)

  return tf.keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)


- 모델 생성

In [26]:
tf.keras.backend.clear_session()

# 하이퍼파라미터
NUM_LAYERS = 5 # 인코더와 디코더의 층의 개수
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수 
UNITS = 256 # 피드 포워드 신경망의 은닉층의 크기
DROPOUT = 0.3 # 드롭아웃의 비율

model = transformer(
    vocab_size=vocab_size,
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)

model.summary()

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, None)]       0           []                               
                                                                                                  
 dec_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 enc_padding_mask (Lambda)      (None, 1, 1, None)   0           ['inputs[0][0]']                 
                                                                                                  
 encoder (Functional)           (None, None, 256)    4059904     ['inputs[0][0]',                 
                                                                  'enc_padding_mask[0][0

- 손실함수

---
레이블인 시퀀스에 패딩이 되어 있으므로, loss를 계산할 때 패딩 마스크를 적용해야 합니다.

In [27]:
def loss_function(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)


- 커스텀 된 학습률

In [28]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):

  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


- 모델 컴파일

In [29]:
learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

- 모델 훈련

In [30]:
EPOCHS = 100
model.fit(dataset, epochs=EPOCHS, verbose=1)

Epoch 1/100
185/185 [==============================] - 41s 127ms/step - loss: 0.9652 - accuracy: 0.0133
Epoch 2/100
185/185 [==============================] - 23s 127ms/step - loss: 0.8013 - accuracy: 0.0200
Epoch 3/100
185/185 [==============================] - 24s 127ms/step - loss: 0.6920 - accuracy: 0.0327
Epoch 4/100
185/185 [==============================] - 24s 127ms/step - loss: 0.6518 - accuracy: 0.0333
Epoch 5/100
185/185 [==============================] - 24s 127ms/step - loss: 0.6249 - accuracy: 0.0340
Epoch 6/100
185/185 [==============================] - 24s 127ms/step - loss: 0.6082 - accuracy: 0.0348
Epoch 7/100
185/185 [==============================] - 24s 127ms/step - loss: 0.5934 - accuracy: 0.0355
Epoch 8/100
185/185 [==============================] - 23s 127ms/step - loss: 0.5796 - accuracy: 0.0361
Epoch 9/100
185/185 [==============================] - 23s 127ms/step - loss: 0.5668 - accuracy: 0.0368
Epoch 10/100
185/185 [==============================] - 24s 127m

## Step 5. 모델 평가하기

Step 1에서 선택한 전처리 방법을 고려하여 입력된 문장에 대해서 대답을 얻는 예측 함수를 만듭니다.

In [31]:
def decoder_inference(sentence):
  sentence = preprocess_sentence(sentence)

  # 입력된 문장을 정수 인코딩 후, 시작 토큰과 종료 토큰을 앞뒤로 추가.
  # ex) Where have you been? → [[8331   86   30    5 1059    7 8332]]
  sentence = tf.expand_dims(
      start_token + tokenizer.encode(sentence) + end_token, axis=0)

  # 디코더의 현재까지의 예측한 출력 시퀀스가 지속적으로 저장되는 변수.
  # 처음에는 예측한 내용이 없음으로 시작 토큰만 별도 저장. ex) 8331
  output_sequence = tf.expand_dims(start_token, 0)

  # 디코더의 인퍼런스 단계
  for i in range(MAX_LENGTH):
    # 디코더는 최대 MAX_LENGTH의 길이만큼 다음 단어 예측을 반복합니다.
    predictions = model(inputs=[sentence, output_sequence], training=False)
    predictions = predictions[:, -1:, :]

    # 현재 예측한 단어의 정수
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    # 만약 현재 예측한 단어가 종료 토큰이라면 for문을 종료
    if tf.equal(predicted_id, end_token[0]):
      break

    # 예측한 단어들은 지속적으로 output_sequence에 추가됩니다.
    # 이 output_sequence는 다시 디코더의 입력이 됩니다.
    output_sequence = tf.concat([output_sequence, predicted_id], axis=-1)

  return tf.squeeze(output_sequence, axis=0)


In [32]:
def sentence_generation(sentence):
  # 입력 문장에 대해서 디코더를 동작 시켜 예측된 정수 시퀀스를 리턴받습니다.
  prediction = decoder_inference(sentence)

  # 정수 시퀀스를 다시 텍스트 시퀀스로 변환합니다.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])

  print('입력 : {}'.format(sentence))
  print('출력 : {}'.format(predicted_sentence))

  return predicted_sentence


- 모델 평가용 문장 리스트

In [33]:
input_question_list = [
    "안녕 오랜만이야",
    "잘 지냈어?",
    "뭐 먹을까",
    "메뉴 추천해줘",
    "어제 뭐 먹었어?",
    "나랑 놀자",
    "어디 가고 싶어?",
    "널 만나서 기뻐",
    "웃어줄래",
    "삶은 뭘까?",
    "인생살이 왜이리 힘드냐",
    "이제 그만 쉬고 싶어",
    "너무 고독하다",
    "죽으면 어떻게 될까",
    "내가 죽으면 슬퍼해 줄거야?",
    "도망가고 싶다",
    "우리 마지막이야",
    "나 간다 잘 지내 안녕",
    "넌 최고였어",
    "나 잊지마",
    "나는 심장이 없어"
]

In [34]:
def print_answers(input_question_list):
    for input_qusetion in input_question_list:
        sentence_generation(input_qusetion)
        print()

In [35]:
print_answers(input_question_list)

입력 : 안녕 오랜만이야
출력 : 지금도 충분해요 .

입력 : 잘 지냈어?
출력 : 마스크 착용 하시고 외출하세요 .

입력 : 뭐 먹을까
출력 : 맛있게 드세요 .

입력 : 메뉴 추천해줘
출력 : 얼른 요리해드세요 .

입력 : 어제 뭐 먹었어?
출력 : 저도 듣고 싶네요 .

입력 : 나랑 놀자
출력 : 조금만 기다리면 다시 전기가 들어올거예요 .

입력 : 어디 가고 싶어?
출력 : 다른 일을 해보세요 .

입력 : 널 만나서 기뻐
출력 : 사랑은 알다가도 모르겠어요 .

입력 : 웃어줄래
출력 : 그 사람을 위해 에너지를 쓰니까요 .

입력 : 삶은 뭘까?
출력 : 서로 더 이해해보세요 .

입력 : 인생살이 왜이리 힘드냐
출력 : 자신의 몸값을 올려보세요 .

입력 : 이제 그만 쉬고 싶어
출력 : 조금만 기다리면 다시 전기가 들어올거예요 .

입력 : 너무 고독하다
출력 : 그럴 때가 있어요 .

입력 : 죽으면 어떻게 될까
출력 : 그럴 때가 있죠 .

입력 : 내가 죽으면 슬퍼해 줄거야?
출력 : 자신의 몸값을 올려보세요 .

입력 : 도망가고 싶다
출력 : 많이 사랑했던 만큼 기억이 선연하더라고요 .

입력 : 우리 마지막이야
출력 : 마음이 아프겠네요 .

입력 : 나 간다 잘 지내 안녕
출력 : 열심히 준비한만큼 잘 볼 수 있을 거예요 .

입력 : 넌 최고였어
출력 : 그럴 때가 있죠 .

입력 : 나 잊지마
출력 : 마음이 아프겠네요 .

입력 : 나는 심장이 없어
출력 : 지금도 늦지 않았어요 .



## 모델 스펙 및 결과

##  1. 첫 시도
### 하이퍼파라미터
BATCH_SIZE : 32  
NUM_LAYERS = 6 # 인코더와 디코더의 층의 개수  
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원  
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수   
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기  
DROPOUT = 0.5 # 드롭아웃의 비율  
EPOCH = 20
### 전처리
중복치를 지워줬음

### 결과
근데 중복치를 지우는 게 문제였는지 챗봇에 귀신이 들려버림..ㅎㅎ  
---
입력 : 안녕 오랜만이야  
출력 : 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 

입력 : 잘 지냈어?  
출력 : 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 

입력 : 뭐 먹을까  
출력 : 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 

입력 : 메뉴 추천해줘  
출력 : 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 

입력 : 어제 뭐 먹었어?  
출력 : 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 

입력 : 나랑 놀자  
출력 : 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 

입력 : 어디 가고 싶어?  
출력 : 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 

더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 

입력 : 나 잊지마  
출력 : 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 


##  2. 두 번째 시도

### 하이퍼파라미터
BATCH_SIZE : 32  
NUM_LAYERS = 6 # 인코더와 디코더의 층의 개수    
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원    
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수   
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기  
DROPOUT = 0.5 # 드롭아웃의 비율  
EPOCH = 50
### 전처리
중복치 제거 과정을 없앤 것 말고는 크게 바뀐 것 없음
### 결과

퇴마 실패.... 전 보다는 정신을 좀 차린 것 같은데 여전히 귀신들림
---
입력 : 안녕 오랜만이야  
출력 : 좋은 될 거예요 .

입력 : 잘 지냈어?  
출력 : 좋은 될 거예요 .

입력 : 뭐 먹을까  
출력 : 좋은 될 거예요 .

입력 : 메뉴 추천해줘  
출력 : 좋은 될 거예요 .

입력 : 어제 뭐 먹었어?  
출력 : 좋은 될 거예요 .

입력 : 나랑 놀자  
출력 : 좋은 될 거예요 .

입력 : 어디 가고 싶어?  
출력 : 좋은 될 거예요 .

입력 : 널 만나서 기뻐  
출력 : 좋은 될 거예요 .

입력 : 웃어줄래  
출력 : 좋은 될 거예요 .

입력 : 삶은 뭘까?  
출력 : 좋은 될 거예요 .

입력 : 인생살이 왜이리 힘드냐  
출력 : 좋은 될 거예요 .

입력 : 이제 그만 쉬고 싶어  
출력 : 좋은 될 거예요 .

## 3. 세 번째 시도

### 하이퍼파라미터
BATCH_SIZE : 32  
MAX_LENGTH = 50  
NUM_LAYERS = 4 # 인코더와 디코더의 층의 개수    
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원    
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수   
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기  
DROPOUT = 0.5 # 드롭아웃의 비율  
EPOCH = 50
### 전처리
이 전에는 [^ㄱ-ㅎㅏ-ㅣ가-힣] 으로 전처리를 했는데 그냥 ㅠㅠ 처럼 돼있는 부분도 다 삭제해버림 [^가-힣]으로 전처리 진행
### 결과

어느 정도는 말이 통하는 듯 하다. 하지만 여전히 이상함. 지금까지는 전처리가 잘못되었던 것 같음. 
---
입력 : 안녕 오랜만이야  
출력 : 너무 걱정하지 마세요 .

입력 : 잘 지냈어?  
출력 : 많이 지쳤나봐요 .

입력 : 뭐 먹을까  
출력 : 맛있는 거 드세요 .

입력 : 메뉴 추천해줘  
출력 : 사랑은 그렇게 모르겠어요 .

입력 : 어제 뭐 먹었어?  
출력 : 얼른 맛난 거 같아요 .

입력 : 나랑 놀자  
출력 : 많이 지쳤나봐요 .

입력 : 어디 가고 싶어?  
출력 : 너무 걱정하지 마세요 .

입력 : 널 만나서 기뻐  
출력 : 너무 걱정하지 마세요 .

입력 : 웃어줄래  
출력 : 너무 걱정하지 마세요 .


## 4. 네 번째 시도

### 하이퍼파라미터
BATCH_SIZE : 64
MAX_LENGTH = 60
NUM_LAYERS = 5 # 인코더와 디코더의 층의 개수  
D_MODEL = 256 # 인코더와 디코더 내부의 입, 출력의 고정 차원  
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수   
UNITS = 256 # 피드 포워드 신경망의 은닉층의 크기  
DROPOUT = 0.3 # 드롭아웃의 비율  
EPOCH = 100

### 결과

---
입력 : 안녕 오랜만이야  
출력 : 지금도 충분해요 .

입력 : 잘 지냈어?  
출력 : 마스크 착용 하시고 외출하세요 .

입력 : 뭐 먹을까  
출력 : 맛있게 드세요 .

입력 : 메뉴 추천해줘  
출력 : 얼른 요리해드세요 .

입력 : 어제 뭐 먹었어?  
출력 : 저도 듣고 싶네요 .

입력 : 나랑 놀자  
출력 : 조금만 기다리면 다시 전기가 들어올거예요 .

입력 : 어디 가고 싶어?  
출력 : 다른 일을 해보세요 .

입력 : 널 만나서 기뻐  
출력 : 사랑은 알다가도 모르겠어요 .

입력 : 웃어줄래  
출력 : 그 사람을 위해 에너지를 쓰니까요 .



## 5. 다섯 번째 시도

### 하이퍼파라미터
BATCH_SIZE : 64  
MAX_LENGTH = 78  
NUM_LAYERS = 4 # 인코더와 디코더의 층의 개수    
D_MODEL = 512 # 인코더와 디코더 내부의 입, 출력의 고정 차원    
NUM_HEADS = 8 # 멀티 헤드 어텐션에서의 헤드 수     
UNITS = 512 # 피드 포워드 신경망의 은닉층의 크기    
DROPOUT = 0.4 # 드롭아웃의 비율    
EPOCH = 200


### 결과

---



입력 : 안녕 오랜만이야  
출력 : 좋은 일이 생길 거예요 .

입력 : 잘 지냈어?  
출력 : 오늘 일찍 주무세요 .

입력 : 뭐 먹을까  
출력 : 맛있는 거 드세요 .

입력 : 메뉴 추천해줘  
출력 : 휴식도 필요하죠 .

입력 : 어제 뭐 먹었어?  
출력 : 같이 가자고 말해보세요

입력 : 나랑 놀자  
출력 : 좋은 생각이에요 .

입력 : 어디 가고 싶어?  
출력 : 저는 자장면이요 .

입력 : 널 만나서 기뻐  
출력 : 저도 간절히 기도 할게요 .

입력 : 웃어줄래  
출력 : 나를 진실로 존중하고 사랑할 때 가능해요 .

## 결과 정리

- 모델 스펙



| 파라미터 | 첫 번째 | 두 번째 | 세 번째 | 네 번째 | 다섯 번째 |
|:---:|:---:|:---:|:---:|:---:|:---:|
|BATCH_SIZE |32 |32|32 |64 | 64|
| MAX_LENGTH|50 |50 | 50| 60| 78|
| NUM_LAYERS| 6| 6| 4|5| 4|
|D_MODEL | 256| 256| 256| 256| 512|
| NUM_HEADS| 8| 8| 8|8 |8 |
|UNITS | 512| 512| 512| 256| 512|
| DROPOUT| 0.5| 0.5| 0.5|0.3 | 0.4|
| EPOCH| 20| 50| 50| 100| 200|


- 출력

| 입력 | 첫 시도 | 두 번째 시도 | 세 번째 시도 | 네 번째 시도 | 다섯 번째 시도 |
|:---:|:---:|:---:|:---:|:---:|:---:|
|안녕 오랜만이야 | 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더  |좋은 될 거예요 .|너무 걱정하지 마세요 .| 지금도 충분해요 .| 좋은 일이 생길 거예요 .|
|잘 지냈어? | 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 |좋은 될 거예요 .| 많이 지쳤나봐요 .| 마스크 착용 하시고 외출하세요 . |오늘 일찍 주무세요 . |
|뭐 먹을까| 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 | 좋은 될 거예요 .| 맛있게 드세요 .| 맛있게 드세요 .|맛있는 거 드세요 . |
| 메뉴 추천해줘|좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더  | 좋은 될 거예요 .| 사랑은 그렇게 모르겠어요 .| 얼른 요리해드세요 .|휴식도 필요하죠 . |
|어제 뭐 먹었어? | 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 |좋은 될 거예요 . |얼른 맛난 거 같아요 . | 저도 듣고 싶네요 .| 같이 가자고 말해보세요|
|나랑 놀자 | 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 |좋은 될 거예요 . | 많이 지쳤나봐요 .| 조금만 기다리면 다시 전기가 들어올거예요 .|좋은 생각이에요 . |
| 어디 가고 싶어?|좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더  |좋은 될 거예요 . |너무 걱정하지 마세요 . | 다른 일을 해보세요 .|저는 자장면이요 . |
|널 만나서 기뻐 | 좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더 |좋은 될 거예요 . |너무 걱정하지 마세요 . |사랑은 알다가도 모르겠어요 . |저도 간절히 기도 할게요 . |
|웃어줄래 |좋은 사람 더 더 더 더 더 더 더 더 더 더 더 더 더 더 더  | 좋은 될 거예요 .| 너무 걱정하지 마세요 .| 그 사람을 위해 에너지를 쓰니까요 .|나를 진실로 존중하고 사랑할 때 가능해요 . |

## 회고

첫 시도랑 두 번째 시도에서 전처리를 할 때 단일 모음으로 이루어진 'ㅜㅜ'같은 것들을 없애지 않고 구두점 분리만 진행 했더니 결과가 너무 이상하게 나왔다. 그리고 첫 시도에서는 end token이 제대로 적용이 안된건지는 모르겠지만 거의 Maxlen만큼 출력이 나왔다... 진짜 처음에 보고 챗봇에 귀신 들린줄 알고 너무 무서웠다 ㅎ;

그래서 괜히 이상한 전처리 하지말고 전처리보단 파라미터 조정에 더 집중하자 해서 기본적으로 LMS에서 진행한 전처리만 진행 해주었고 파라미터만 조정을 하니까 퇴마에 성공했다 ㅋ_ㅋ

그래도 여전히 이상한 답을 내놓긴 하지만 처음보단 많이 나아졌으니 뭐....

이번 EX는 챗봇 생성도 중요하겠지만 Transformer 구조에 대해서 공부하는 게 더 중요하다는 생각이 든다. 코드를 직접 치면서도 이해가 안되는 부분이 많아서 기계적인 타이핑만 거의 했는데 LMS내용을 두고두고 보면서 Transformer의 근본에 대해서 공부를 좀 더 해야겠다.
